## Assignment 5

# Part 1: Heterogeneous treatment effects using causal trees and forests
For this part, we will be using experimental data for computing heterogeneous effects through causal trees and forests. For all exercises, the predictors $X$ are all variables that are not the outcome $Y$ or the treatment $D$.

### 1.1. Load the data (1 points). 
This is data for and experiment regarding the National Supported Work Demonstration (NSW) job-training program. You can find the data here, and read a description of the data here. For further details of the experiment and the program, you can use this link

In [2]:
using MLJ, MLJModels, RDatasets, DataFrames
using CSV, DataFrames, StatsModels, GLM, Random, RData, MLBase, MLJ, PrettyTables, FixedEffectModels

In [2]:
# Leer el archivo CSV
df = CSV.read("/Users/gabriel/Documents/GitHub/CausalAI-Course/Labs/Assignment/Assignment_5/data/experimental/experimental_control.csv", DataFrame)
df

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,1,37,11,1,0,1,1,0.0,0.0,9930.05
2,1,22,9,0,1,0,1,0.0,0.0,3595.89
3,1,30,12,1,0,0,0,0.0,0.0,24909.4
4,1,27,11,1,0,0,1,0.0,0.0,7506.15
5,1,33,8,1,0,0,1,0.0,0.0,289.79
6,1,22,9,1,0,0,1,0.0,0.0,4056.49
7,1,23,12,1,0,0,0,0.0,0.0,0.0
8,1,32,11,1,0,0,1,0.0,0.0,8472.16
9,1,22,16,1,0,0,0,0.0,0.0,2164.02


### 1.2. Find the ATE (1.5 points). 
With `re78` as the outcome variable of interest, find the Average Treatment Effect of participation in the program. Specifically, you should find it by calculating the difference between the means of the treatment group and the control group (the Simple Difference of Means or SDM). What can you say about the program?

In [22]:
using GLM
using DataFrames


# Fit the linear model
lin_model = lm(@formula(re78 ~ treat), df)

# Display the summary of the model
lin_model

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

re78 ~ 1 + treat

Coefficients:
───────────────────────────────────────────────────────────────────────
               Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────
(Intercept)  4554.8      408.046  11.16    <1e-24   3752.85     5356.75
treat        1794.34     632.853   2.84    0.0048    550.574    3038.11
───────────────────────────────────────────────────────────────────────

### 1.3. Heterogeneous effects with causal trees (3 points). 
Use causal trees like we saw in class. For Python, you should use the `econml` package; for R, use the `grf` package; and for Julia, you will need to create the auxiliary variable $Y$∗ and fit a decision tree regressor. Report the splits the tree finds and interpret them.

In [ ]:
using MLJ 

#Acquire our target vector and our feature matrix.
y, X = unpack(df, ==(:re78), !=(:re78));
coerce!(X, Textual => Multiclass)
D, X = unpack(X, ==(:treat), !=(:treat));

In [ ]:
coerce!(X, Count => Continuous);

In [43]:
using MLJ, MLJModels, RDatasets, DataFrames

LogisticClassifier = @load LogisticClassifier pkg=MLJScikitLearnInterface verbosity=0

log_model = LogisticClassifier()

log_model_machine = machine(log_model, X, D)

fit!(log_model_machine)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc MLJScikitLearnInterface.LogisticClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{AbstractVector{Continuous}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @ MLJBase ~/.julia/packages/MLJBase/7nGJF/src/machines.jl:237
[ Info: Training machine(LogisticClassif

LoadError: DomainError with 1:
Can only convert categorical elements to integers. 

In [27]:
LogisticClassifier = @load LogisticClassifier pkg=MLJScikitLearnInterface #regresion logistica

log_model = LogisticClassifier()

log_model_machine = machine(log_model,X, D) #maquina que da acceso ala interfaz de 

fit!(log_model_machine)

[ Info: For silent loading, specify `verbosity=0`. 


import MLJScikitLearnInterface ✔


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc MLJScikitLearnInterface.LogisticClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{14}}, AbstractVector{Multiclass{2}}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @ MLJBase ~/.julia/packa

trained Machine; caches model-specific representations of data
  model: LogisticClassifier(penalty = l2, …)
  args: 
    1:	Source @691 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Multiclass{14}}, AbstractVector{Multiclass{2}}}}
    2:	Source @755 ⏎ AbstractVector{Multiclass{2}}


In [ ]:
DecisionTreeRegressor = (@load DecisionTreeRegressor pkg=DecisionTree verbosity=0)
tree_model = DecisionTreeRegressor(max_depth = 2)
tree_machine = machine(tree_model, X, y_star)
fit!(tree_machine);

In [ ]:
fitted_params(tree_machine)[1]

### 1.4. Heterogeneous effects with causal forests (3 points). 
Use causal forests like we saw in class. For Python, you should use the `econml` package; for R, use the `grf` package; and for Julia, you will need to use the auxiliary variable $Y$ ∗ computed in the previous exercise and fit a random forest regressor. Report the importance of the prediction variables.

### 1.5. Plot heterogeneous effects (1.5 points). 
Plot how the predicted treatment effect changes depending on a variable of your choice. (You can see the last example in PD11 for clarification of what you should do in this exercise)

# Part 2: Double/Debiased machine learning in observational data
In this part, we will be using observational data for computing the average treatment effect of the same program as in Part 1. This data is constructed by taking the treatment group from the same dataset as in Part 1, but constructing the control group from a different dataset; that is, the entirety of the control is comprised of observations from the Current Population Survey. Therefor, we may not have comparable treatment and control groups. To tackle this issue, we can use Double/Debiased machine learning.

### 2.1. Load the data (1 points). 
You can find the data here, and read a description of the data here. For further details on how this data was created, you can use this link.

In [3]:
df2 = CSV.read("/Users/gabriel/Documents/GitHub/CausalAI-Course/Labs/Assignment/Assignment_5/data/observational/biased_control.csv", DataFrame)
coerce!(df2, Count => Continuous);

### 2.2. Group comparisons (1.5 points). 
For the treatment and control group separately, report summary statistics of three variables of your choice. Can you spot any big differences between the treatment and control groups?

In [ ]:
# Agrupar el DataFrame por la columna 'treat'
grouped = groupby(df2, :treat)

control = grouped[1]
treat = grouped[2]
control = DataFrame(control)
treat = DataFrame(treat)

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,0.0,9.93005,1369.0,50653.0,121.0,1.0,1.0,0.0,0.0,0.0
2,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,0.0,3.59589,484.0,10648.0,81.0,1.0,1.0,0.0,0.0,0.0
3,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,24.9094,900.0,27000.0,144.0,1.0,1.0,0.0,0.0,0.0
4,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,7.50615,729.0,19683.0,121.0,1.0,1.0,0.0,0.0,0.0
5,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,0.0,0.28979,1089.0,35937.0,64.0,1.0,1.0,0.0,0.0,0.0
6,1.0,22.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,4.05649,484.0,10648.0,81.0,1.0,1.0,0.0,0.0,0.0
7,1.0,23.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,529.0,12167.0,144.0,1.0,1.0,0.0,0.0,0.0
8,1.0,32.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,8.47216,1024.0,32768.0,121.0,1.0,1.0,0.0,0.0,0.0
9,1.0,22.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,2.16402,484.0,10648.0,256.0,1.0,1.0,0.0,0.0,0.0


In [5]:
# Calcular estadísticas descriptivas para un DataFrame
function calc_stats(df::DataFrame)
    stats = DataFrame()  # Crear un DataFrame vacío para almacenar las estadísticas
    
    for col in names(df)
        # Evitar cálculo si la columna no es numérica
        if eltype(df[!, col]) <: Number
            # Calcular las estadísticas por columna
            mean_val = mean(skipmissing(df[!, col]))
            sd_val = std(skipmissing(df[!, col]))
            min_val = minimum(skipmissing(df[!, col]))
            max_val = maximum(skipmissing(df[!, col]))
            
            # Crear un NamedTuple para cada columna con los resultados
            row = (variable=col, mean=mean_val, sd=sd_val, min=min_val, max=max_val)
            
            # Agregar la fila como NamedTuple
            push!(stats, row)
        end
    end
    return stats
end

# Convertir el SubDataFrame a DataFrame antes de pasar a calc_stats
control_df = DataFrame(control)
treat_df = DataFrame(treat)

# Calcular estadísticas descriptivas para el grupo 'control'
control_stats = calc_stats(control_df[:, [:agesq, :educsq, :u75]])

# Calcular estadísticas descriptivas para el grupo 'treat'
treat_stats = calc_stats(treat_df[:, [:agesq, :educsq, :u75]])

# Mostrar los resultados
println("\nNúmero de observaciones de control: ", nrow(control))
println(control_stats)
println("\nNúmero de observaciones de tratamiento: ", nrow(treat))
println(treat_stats)



Número de observaciones de control: 15992
3×5 DataFrame
 Row │ variable  mean         sd          min      max     
     │ String    Float64      Float64     Float64  Float64 
─────┼─────────────────────────────────────────────────────
   1 │ agesq     1225.91      784.738       256.0   3025.0
   2 │ educsq     152.902      67.1663        0.0    324.0
   3 │ u75          0.109305    0.312031      0.0      1.0

Número de observaciones de tratamiento: 185
3×5 DataFrame
 Row │ variable  mean     sd          min      max     
     │ String    Float64  Float64     Float64  Float64 
─────┼─────────────────────────────────────────────────
   1 │ agesq     717.395  431.252       289.0   2304.0
   2 │ educsq    111.059   39.3039       16.0    256.0
   3 │ u75         0.6      0.491227      0.0      1.0


##### There are a large number of observations in the control group compared to the treatment group. We find that the age and years of education of the control group are on average much higher than those of the treatment group, which contains younger and less educated individuals. We also see that the probability of individuals in the treatment group being unemployed in 1975 is much higher than that of the control individuals.

### 2.3. Compute the SMD (1.5 points). 
Find the simple difference of means, which we can use as a naive estimate of the ATE. How does the result in this case compare to the result in point 1.2.?

In [ ]:
# normalización
numeric_columns = [:age, :educ, :re74, :re75, :re78, :agesq, :agecube, :educsq, :interaction1, :re74sq, :re75sq]
x_numeric = select(df2, numeric_columns)

for col in numeric_columns
    df2[!, col] = (df2[!, col] .- mean(df2[!, col])) ./ std(df2[!, col])
end

first(df2, 5) 

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,1.07457,-0.351563,0.0,0.0,1.0,1.0,0.794352,1.25965,1.11831,1.02738,0.935689,-0.46859,0.0,0.0,0.518815,0.682789,1.48892
2,0.0,-1.10003,0.694461,0.0,0.0,0.0,0.0,-1.11353,-0.82245,-0.129604,-0.994422,-0.882194,0.649806,0.0,0.0,-0.964524,-1.05545,-0.951227
3,0.0,0.440311,-0.0028882,0.0,0.0,1.0,0.0,0.952714,1.24753,1.11831,0.285796,0.13065,-0.125615,0.0,0.0,0.825831,0.943132,1.46592
4,0.0,1.34639,-2.09494,0.0,0.0,1.0,1.0,1.15612,1.25965,1.11831,1.38349,1.36798,-1.73611,0.0,0.0,-0.150372,1.30362,1.48892
5,0.0,-1.37186,-1.39759,0.0,0.0,1.0,1.0,-1.27026,-0.298995,-0.505492,-1.14376,-0.958339,-1.31857,0.0,0.0,-1.20459,-1.08347,-0.624129


In [10]:
y = df2[!,"re78"];
d = df2[!,"treat"];
x = df2[!, Not([:"treat", :"re78"])];

In [12]:
# Diferencia en medias
mean_treated = mean(filter(:treat => ==(1), df2)[!,"re78"])
mean_control = mean(filter(:treat => ==(0), df2)[!,"re78"])

# ATE (Average Treatment Effect)
ate = mean_treated - mean_control

println("Mean re78 for the treated group: $(round(mean_treated, digits=2))")
println("Mean re78 for the control group: $(round(mean_control, digits=2))")
println("Average Treatment Effect (ATE): $(round(ate, digits=2))")

Mean re78 for the treated group: -0.87
Mean re78 for the control group: 0.01
Average Treatment Effect (ATE): -0.88


##### The results are vastly different because this dataset comes from an observational sample that comes from other different datasets, which makes it impossible to compare both groups of data. We will estimate a double/biased machine learning to find the unobservable effects of the treatment group on the control group to perform causal inference of the work program.

### 2.4. Using DML (6 points). 
Use the DML procedure as we saw in the Lab, in order to find a better estimate of the ATE. You may use the `doubleML` packages for Python and R, but this package does not exist for Julia, so you will have to build your own procedure like we saw in class. You will be rewarded extra points for using more than one method for predictions. At the end, report the treatment effect you found, as well as the MSE for $D$ and $Y$ achieved by the method(s) you used.

In [14]:
function training_sample_append(cv_split, test_sample_index)
    training_indices = []
    for vector in cv_split[Not(test_sample_index)]
            training_indices = [training_indices; vector]
    end
    return training_indices, cv_split[test_sample_index]
end

function dml(x, d, y, dreg, yreg, nfold)
    n = length(y)
    cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
    machine_y = machine(yreg, x, y, scitype_check_level=0)
    machine_d = machine(dreg, x, d, scitype_check_level=0)
    y_hat = zeros(n)
    d_hat = zeros(n)

    for fold in 1:nfold
            training_fold, test_fold = training_sample_append(cv, fold)
            y_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_y, rows = training_fold), x[test_fold, :])
            d_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_d, rows = training_fold), x[test_fold, :])
    end

    resy = y .- y_hat
    resd = reshape(d .- d_hat, (n, 1))
    estimate = lm(resd, resy)
    coef_est = GLM.coef(estimate)[1]
    se = GLM.coeftable(estimate).cols[2][1]
    println(" coef (se) = ", coef_est ,"(",se,")")
    return coef_est, se, resy, resd;
end

function summarize(point, stderr, resy, resd, name)
    return DataFrame(
            model = [name],
            estimate = [point], stderr = [stderr], 
            rmse_y = [sqrt(mean(resy .^ 2))], 
            rmse_d = [sqrt(mean(resd .^ 2))]
    )
end

summarize (generic function with 1 method)

In [ ]:
# Pkg.add("MLJScikitLearnInterface")
# Pkg.add("LIBSVM")
# Pkg.add("NearestNeighborModels")
# Pkg.add("MLJXGBoostInterface")

   Resolving package versions...
   Installed CUDA_Runtime_jll ──── v0.15.5+0
   Installed CUDA_Driver_jll ───── v0.10.4+0
   Installed MLJXGBoostInterface ─ v0.3.12
   Installed XGBoost_jll ───────── v2.0.1+0
   Installed SparseMatricesCSR ─── v0.6.8
   Installed XGBoost ───────────── v2.5.1
    Updating `~/.julia/environments/v1.10/Project.toml`
  [54119dfa] + MLJXGBoostInterface v0.3.12
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [54119dfa] + MLJXGBoostInterface v0.3.12
  [a0a7dd2c] + SparseMatricesCSR v0.6.8
  [009559a3] + XGBoost v2.5.1
  [4ee394cb] + CUDA_Driver_jll v0.10.4+0
  [76a88914] + CUDA_Runtime_jll v0.15.5+0
  [a5c6f535] + XGBoost_jll v2.0.1+0
Precompiling project...
  ✓ SparseMatricesCSR
  ✓ CUDA_Driver_jll
  ✓ CUDA_Runtime_jll
  ✓ XGBoost_jll
  ✓ XGBoost
  ✓ MLJXGBoostInterface
  6 dependencies successfully precompiled in 11 seconds. 382 already precompiled.


In [15]:
# OLS
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg_ols = Standardizer() |> LinearRegressor()
yreg_ols = Standardizer() |> LinearRegressor()
result_ols = dml(x, d, y, dreg_ols, yreg_ols, 10)
table_ols = summarize(result_ols..., "OLS")

[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).


 coef (se) = 0.16243815033833317(0.05755949821175117)


[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,OLS,0.162438,0.0575595,0.722501,0.0986685


In [16]:
# Lasso
LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg_lasso = Standardizer() |> LassoCVRegressor(max_iter=200000)
yreg_lasso = Standardizer() |> LassoCVRegressor(max_iter=200000)
result_lasso = dml(x, d, y, dreg_lasso, yreg_lasso, 10)
table_lasso = summarize(result_lasso..., "Lasso")

[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regre

 coef (se) = 0.1504855951892169(0.05750272071593801)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,Lasso,0.150486,0.0575027,0.722819,0.0988129


In [17]:
# Random Forest
RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg_rf = RandomForestRegressor();
yreg_rf = RandomForestRegressor();
result_rf = dml(x, d, y, dreg_rf, yreg_rf, 10);
table_rf = summarize(result_rf..., "RF")

[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(

 coef (se) = 0.14891393122936214(0.06650655988736476)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,RF,0.148914,0.0665066,0.765271,0.0904581


In [ ]:
# Elastic Net
ElasticNetRegressor = @load ElasticNetRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg_en = Standardizer() |> ElasticNetRegressor(alpha=0.1, l1_ratio=0.1)
yreg_en = Standardizer() |> ElasticNetRegressor(alpha=0.1, l1_ratio=0.1)
result_en = dml(x, d, y, dreg_en, yreg_en, 10)
table_en = summarize(result_en..., "ElasticNet")

[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:elastic_net_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:elastic_net_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:elastic_net_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:elastic_net_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:el

 coef (se) = 0.07071735434431879(0.056475437130961946)


[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:elastic_net_regressor, …).


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,ElasticNet,0.0707174,0.0564754,0.726553,0.101147


In [19]:
# K Nearest Neighbors
KNNRegressor = @load KNNRegressor pkg=NearestNeighborModels verbosity=0
dreg = KNNRegressor()
yreg = KNNRegressor()
results_knn = dml(x, d, y, dreg, yreg, 10)
table_knn = summarize(results_knn..., "KNN")

[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …), …).
[ Info: Training machine(KNNRegressor(K = 5, …

 coef (se) = 0.05015451710251798(0.06893063051309897)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,KNN,0.0501545,0.0689306,0.784709,0.0895063


In [20]:
# Gradient Boosting Machine (GBM) using XGBoost
XGBoostRegressor = @load XGBoostRegressor pkg=XGBoost verbosity=0
dreg_gbm = XGBoostRegressor(
    num_round=100,      # Número de iteraciones
    max_depth=3,        # Profundidad máxima de los árboles
    eta=0.1,            # Tasa de aprendizaje
    booster="gbtree"    # Modelo base (árboles)
)
yreg_gbm = XGBoostRegressor(
    num_round=100, 
    max_depth=3, 
    eta=0.1, 
    booster="gbtree"
)

result_gbm = dml(x, d, y, dreg_gbm, yreg_gbm, 10)
table_gbm = summarize(result_gbm..., "GBM")


[ Info: Training machine(XGBoostRegressor(test = 1, …), …).
[ Info: XGBoost: starting training.
[ Info: [1]	train-rmse:1.05391655913851578
[ Info: [2]	train-rmse:1.00138213658844211
[ Info: [3]	train-rmse:0.95662603212740116
[ Info: [4]	train-rmse:0.91806189673438454
[ Info: [5]	train-rmse:0.88566999579224326
[ Info: [6]	train-rmse:0.85822220243397573
[ Info: [7]	train-rmse:0.83484569593245017
[ Info: [8]	train-rmse:0.81553268279949542
[ Info: [9]	train-rmse:0.79939764935022095
[ Info: [10]	train-rmse:0.78561582254571205
[ Info: [11]	train-rmse:0.77438017538093462
[ Info: [12]	train-rmse:0.76457170876687341
[ Info: [13]	train-rmse:0.75666789410659219
[ Info: [14]	train-rmse:0.74976424719014156
[ Info: [15]	train-rmse:0.74397584597225108
[ Info: [16]	train-rmse:0.73927689041242850
[ Info: [17]	train-rmse:0.73534207840218602
[ Info: [18]	train-rmse:0.73187683805669146
[ Info: [19]	train-rmse:0.72914958401811059
[ Info: [20]	train-rmse:0.72677959102378942
[ Info: [21]	train-rmse:0.7247634

 coef (se) = 0.09312162556102348(0.06671856824782658)


[ Info: [100]	train-rmse:0.08185142022684970
[ Info: Training rounds complete.


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,GBM,0.0931216,0.0667186,0.72033,0.0848835


In [22]:
# Combinar todos los resultados en una tabla
pretty_table([table_ols; table_lasso; table_rf; table_en; table_knn; table_gbm])

┌────────────┬───────────┬───────────┬──────────┬───────────┐
│      model │  estimate │    stderr │   rmse_y │    rmse_d │
│     String │   Float64 │   Float64 │  Float64 │   Float64 │
├────────────┼───────────┼───────────┼──────────┼───────────┤
│        OLS │  0.162438 │ 0.0575595 │ 0.722501 │ 0.0986685 │
│      Lasso │  0.150486 │ 0.0575027 │ 0.722819 │ 0.0988129 │
│         RF │  0.148914 │ 0.0665066 │ 0.765271 │ 0.0904581 │
│ ElasticNet │ 0.0707174 │ 0.0564754 │ 0.726553 │  0.101147 │
│        KNN │ 0.0501545 │ 0.0689306 │ 0.784709 │ 0.0895063 │
│        GBM │ 0.0931216 │ 0.0667186 │  0.72033 │ 0.0848835 │
└────────────┴───────────┴───────────┴──────────┴───────────┘


##### For this case, the Gradient Boosting Machine model predicts the outcome and treatment variables much better. This may be mainly due to the fact that it depends on a loss function that may be using the RMSE as a loss metric, which would indicate greater power to combat bias.

##### Also, since this model also depends on decision trees, but with loss functions, we can say that according to the results, there is a non-linear relationship between the treatment variables, given the set of co-variables, which is evidenced by a better performance, unlike other linear models such as OLS, lasso and ElasticNet.

##### Finally, we find that under the GBM model the aggregate causal effect is 0.0931.